In [228]:
# Dependencies.
import pandas as pd
from config import client_id, client_secret
from params import key_signatures, modes, key_int
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from pprint import pprint
import numpy as np

# Load variables to environment.
os.environ['SPOTIPY_CLIENT_ID'] = client_id
os.environ['SPOTIPY_CLIENT_SECRET'] = client_secret

sp = spotipy.Spotify(client_credentials_manager = SpotifyClientCredentials())

In [229]:
# Function to obtain user's input.
def get_input():

    user_input = input('Enter the playlist URI: ')

    return user_input

In [230]:
# Function to perform API call and return results in JSON format.
def get_results(t, o):

    # Check if string contains 'https', the ref['next'] will contain this if the list has more than 100 songs.
    if 'https' in t:

        # Split the string in URL using '/'
        split = t.split('/')

        # Split list index 5 will contain part of URI, multiple o by 100 for the offset in the function call.
        results = sp.playlist_tracks('spotify:playlist:' + split[5], offset = o * 100)

    else:
        results = sp.playlist_tracks(t)

    return results

In [231]:
# Function to create dict with data.
def create_df(results):

    # Empty dict to hold data from Spotify.
    data = {
        'track': [],
        'artist': [],
        'year': [],
        'uri': []
    }

    # Loop control.
    loop = True

    # Results as json object.
    ref = results

    # Offset variable.
    offset = 0

    while loop:

        # Adds values to dict.
        for song in ref['items']:
            data['track'].append(song['track']['name'])
            data['artist'].append(song['track']['artists'][0]['name'])
            data['year'].append(get_year(song['track']))
            data['uri'].append(song['track']['uri'])

        # Check for ref['next'], if there's text then playlist has more items.
        if ref['next'] is not None:
            offset += 1
            ref = get_results(ref['next'], offset)
        
        # Terminate loop.
        else:
            loop = False

    return pd.DataFrame(data)

In [232]:
# Function to perform API call for audio features and return results in JSON format.
def get_features(t):

    results = sp.audio_features(t)

    return results

In [233]:
# Function to find the time key signatures of the song.
def features_finder(results):

    try:

        m = results[0]['mode']

        k = results[0]['key']

        t = results[0]['time_signature']

        b = results[0]['tempo']
    
    except Error:
        
        return np.NaN, np.NaN, np.NaN, np.NaN
    
    return m, k, t, b

In [234]:
# Function to return the year of release as an int.
def get_year(t):

    results = sp.album(t['album']['uri'])
    date = results['release_date']
    year = int(date[0:4])
    return year

In [235]:
results = get_results(get_input(), 1)
df = create_df(results)

uris = list(df['uri'])

md_col = []
ks_col = []
kint_col = []
ts_col = []
bpm_col = []

for uri in uris:
    response = get_features(uri)
    md, ks, ts, bpm = features_finder(response)

    md_col.append(md)
    ks_col.append(ks)
    kint_col.append(key_int[md][ks])
    ts_col.append(ts)
    bpm_col.append(bpm)


df['mode'] = md_col
df['key_signature'] = ks_col
df['key_integer'] = kint_col
df['time_signature'] = ts_col
df['bpm'] = bpm_col


df = df.dropna()
df

,track,artist,year,uri,mode,key_signature,key_integer,time_signature,bpm
0,Walk,Pantera,1992,spotify:track:7fcfNW0XxTWlwVlftzfDOR,0,1,4,4,118.242
1,Master Of Puppets,Metallica,1986,spotify:track:2MuWTIM3b0YEAskbeeFE1i,0,4,1,4,105.173
2,Raining Blood,Slayer,1986,spotify:track:4fiOTntQKr24p07FvQDHZE,0,10,-5,4,87.499
3,Duality,Slipknot,2004,spotify:track:61mWefnWQOLf90gepjOCb3,0,4,1,4,143.772
4,The Trooper - 2015 Remaster,Iron Maiden,1983,spotify:track:4OROzZUy6gOWN4UGQVaZMF,0,4,1,4,159.756
...,...,...,...,...,...,...,...,...,...
72,The Sound Of Truth,As I Lay Dying,2007,spotify:track:7lYrz70DntWR7o8Sqfz0Ok,0,2,-1,4,165.018
73,1000hp,Godsmack,2014,spotify:track:3NW1h8l6EY5N6HNBL83S4V,0,6,3,4,92.091
74,The American Way,Sacred Reich,1990,spotify:track:2SA2tTWoELC0ciMuYRfbpT,1,0,0,4,148.524
75,Elimination,Overkill,1989,spotify:track:6lWFQo5NuR3y8Q6CHIRatA,0,6,3,4,112.766


In [236]:
df.to_csv('output.csv', index = False)